In [1]:
import os
import cv2
from PIL import Image
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm

import time
import timeit

C:\Users\aquat\AppData\Local\Temp\ipykernel_12748\321412031.py:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
import keras
import tensorflow as tf
from keras import layers
from keras.models import Model
from keras.layers import Lambda, concatenate
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D , Conv2D , MaxPooling2D
from keras.layers import  Dropout , BatchNormalization , Dense
from keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from keras.callbacks import Callback , ReduceLROnPlateau , ModelCheckpoint
from sklearn.metrics import cohen_kappa_score, accuracy_score
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy
     

In [3]:
# Loading Dataset

nonDemented_train = np.load('/CSE499/non_demented_train.npy')
nonDemented_test = np.load('/CSE499/non_demented_test.npy')

veryMildDemented_train = np.load('/CSE499/very_mild_demented_train.npy')
veryMildDemented_test = np.load('/CSE499/very_mild_demented_test.npy')

mildDemented_train = np.load('/CSE499/mild_demented_train.npy')
mildDemented_test = np.load('/CSE499/mild_demented_test.npy')

moderateDemented_train = np.load('/CSE499/moderate_demented_train.npy')
moderateDemented_test = np.load('/CSE499/moderate_demented_test.npy')

In [4]:
# Shape of our dataset
print(f'Non Demented Train: ',nonDemented_train.shape)
print(f'Very Mild Demented Train: ',veryMildDemented_train.shape)
print(f'Mild Demented Train: ',mildDemented_train.shape)
print(f'Moderate Demented Train: ',moderateDemented_train.shape)

print(f'Non Demented Test: ',nonDemented_test.shape)
print(f'Very Mild Demented Test: ',veryMildDemented_test.shape)
print(f'Mild Demented Test: ',mildDemented_test.shape)
print(f'Moderate Demented Test: ',moderateDemented_test.shape)

Non Demented Train:  (2560, 128, 128, 1)
Very Mild Demented Train:  (1792, 128, 128, 1)
Mild Demented Train:  (716, 128, 128, 1)
Moderate Demented Train:  (51, 128, 128, 1)
Non Demented Test:  (640, 128, 128, 1)
Very Mild Demented Test:  (448, 128, 128, 1)
Mild Demented Test:  (180, 128, 128, 1)
Moderate Demented Test:  (13, 128, 128, 1)


In [40]:
from sklearn.utils import shuffle

# Assuming you have loaded and split your data into X_train, Y_train, X_test, and Y_test

# Initialize empty lists to hold data and labels
data_train = []
labels_train = []
data_test = []
labels_test = []

# For the "Non_Demented" class
data_train.extend(nonDemented_train)
labels_train.extend([0] * len(nonDemented_train))
data_test.extend(nonDemented_test)
labels_test.extend([0] * len(nonDemented_test))

# For the "Very_Mild_Demented" class
data_train.extend(veryMildDemented_train)
labels_train.extend([1] * len(veryMildDemented_train))
data_test.extend(veryMildDemented_test)
labels_test.extend([1] * len(veryMildDemented_test))

# For the "Mild_Demented" class
data_train.extend(mildDemented_train)
labels_train.extend([2] * len(mildDemented_train))
data_test.extend(mildDemented_test)
labels_test.extend([2] * len(mildDemented_test))

# For the "Moderate_Demented" class
data_train.extend(moderateDemented_train)
labels_train.extend([3] * len(moderateDemented_train))
data_test.extend(moderateDemented_test)
labels_test.extend([3] * len(moderateDemented_test))

# Convert lists to NumPy arrays
X_train = np.array(data_train)
Y_train = np.array(labels_train)
X_test = np.array(data_test)
Y_test = np.array(labels_test)

# Shuffle the data and labels
X_train, Y_train = shuffle(X_train, Y_train, random_state=42)
X_test, Y_test = shuffle(X_test, Y_test, random_state=42)

# Convert labels to one-hot encoded format
Y_train = to_categorical(Y_train, num_classes=4)
Y_test = to_categorical(Y_test, num_classes=4)


In [41]:
print(f'X train shape:',X_train.shape)
print(f'X test shape:',X_test.shape)
print(f'Y train shape:',Y_train.shape)
print(f'Y test shape:',Y_test.shape)

X train shape: (5119, 128, 128, 1)
X test shape: (1281, 128, 128, 1)
Y train shape: (5119, 4)
Y test shape: (1281, 4)


In [42]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=10)

In [43]:
print(f'x train shape:',x_train.shape)
print(f'x val shape:',x_val.shape)
print(f'y train shape:',y_train.shape)
print(f'y val shape:',y_val.shape)

x train shape: (4095, 128, 128, 1)
x val shape: (1024, 128, 128, 1)
y train shape: (4095, 4)
y val shape: (1024, 4)


In [44]:
# Teacher model with ResNet50

def build_model(backbone, lr=1e-4):
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(4, activation='softmax'))  # Adjusted for 4 classes
    
    model.compile(
        loss='categorical_crossentropy',  # Adjusted for multi-class classification
        optimizer=Adam(learning_rate=lr),
        metrics=['accuracy']
    )
    
    return model

# Load ResNet50 with weights pre-trained on ImageNet
resnet = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(128, 128, 3)
)

# Call the model with the modified settings
model = build_model(resnet, lr=1e-4, num_classes=4)
model.build((None, 128, 128, 3))
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d_5   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_5 (Dropout)         (None, 2048)              0         
                                                                 
 batch_normalization_5 (Batc  (None, 2048)             8192      
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 4)                 8196      
                                                                 
Total params: 23,604,100
Trainable params: 23,546,884


In [45]:
learn_control = ReduceLROnPlateau(monitor='val_acc', patience=5,verbose=1,factor=0.2, min_lr=1e-4)
filepath="ResNet50weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [46]:
# Training the teacher model with Resnet50
# Train the teacher model as usual
# Configuration
import time
epochs = 10 
batch_size = 64

# Calculate the starting time    
start_time = time.time()

teacher_his = model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_val, y_val),
            callbacks=[learn_control, checkpoint])


end_time = time.time()
print("--- Time taken to train : %s seconds ---" % ((end_time - start_time)))

Epoch 1/10


ValueError: in user code:

    File "c:\Users\aquat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\aquat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\aquat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\aquat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\aquat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\aqua